# 초기화

In [2]:
import pymongo

MONGO_URI = "mongodb://localhost:27017/"

try:
    # Connect to MongoDB client
    client = pymongo.MongoClient(MONGO_URI)
    database_names = client.list_database_names()
    print(f"\nCurrent databases: {database_names}")
    
except pymongo.errors.ConnectionFailure as e:
    print(f"Connection error: {e}")


Current databases: ['admin', 'config', 'local', 'log_database', 'test_database']


In [3]:
# System databases to be skipped
system_databases = ["admin", "config", "local"]

if database_names:
    print("\n Deleting all non-system databases...")
    for db_name in database_names:
        if db_name not in system_databases:
            client.drop_database(db_name)
            print(f" Database '{db_name}' has been deleted.")
        else:
            print(f" Skipping system database '{db_name}'.")

    # Verify deletion by listing databases again
    remaining_databases = client.list_database_names()
    print(f"\n Databases remaining: {remaining_databases}")

# Close the connection
client.close()
print("\n Connection closed. All operations completed.")


 Deleting all non-system databases...
 Skipping system database 'admin'.
 Skipping system database 'config'.
 Skipping system database 'local'.
 Database 'log_database' has been deleted.
 Database 'test_database' has been deleted.

 Databases remaining: ['admin', 'config', 'local']

 Connection closed. All operations completed.


# 쉬운 예제

In [4]:
from datetime import datetime, timedelta, timezone
from pymongo import MongoClient, ASCENDING, DESCENDING
from pprint import pprint

client = MongoClient("mongodb://127.0.0.1:27017")

# test_database 생성
db = client['test_database']
print(db)

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')


In [5]:
# hello_collection 생성
msg_collection = db['hello_collection']

# Hello, World! 텍스트 문서 생성
document = {
    "message" : "Hello, World!", 
    "timestamp" : datetime.now()
}

# 문서 삽입 
result = msg_collection.insert_one(document)
print(f"문서삽입 성공, ID: {result.inserted_id}")

문서삽입 성공, ID: 68a5646c6d0b2fb118f138d8


In [6]:
# 저장된 문서 확인 
for doc in msg_collection.find():
    print(doc)

{'_id': ObjectId('68a5646c6d0b2fb118f138d8'), 'message': 'Hello, World!', 'timestamp': datetime.datetime(2025, 8, 20, 15, 0, 12, 542000)}


# 에러 로그 파일

## 모든 함수 정의

In [7]:
import pymongo
import re
import json

def parse_error_logs(file_path):
    """
    error_logs.txt 파일을 파싱하여 JSON 문서 리스트로 반환합니다.
    """
    logs = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            if "BROKEN LINE WITHOUT FIELDS" in line:
                logs.append({"log_type": "corrupted", "message": line})
            elif re.match(r'^".*" \d{3}$', line):
                match = re.match(r'^"(.+)" (\d{3})$', line)
                logs.append({
                    "log_type": "incomplete",
                    "user_agent": match.group(1),
                    "status_code": int(match.group(2))
                })
            else:
                match = re.match(r'^(\S+) - (\S+) \[([^\]]+)\] "([^"]+)" (\d{3}) (\S+) "([^"]*)" "([^"]*)"$', line)
                if match:
                    logs.append({
                        "log_type": "full",
                        "ip": match.group(1),
                        "user": match.group(2),
                        "timestamp": match.group(3),
                        "request": match.group(4),
                        "status_code": int(match.group(5)),
                        "size": int(match.group(6)) if match.group(6).isdigit() else match.group(6),
                        "referer": match.group(7) if match.group(7) != '-' else None,
                        "user_agent": match.group(8) if match.group(8) != '-' else None
                    })
    return logs

def parse_advertools_logs(file_path):
    """
    advertools_error_logs.txt 파일을 파싱하여 JSON 문서 리스트로 반환합니다.
    """
    logs = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.split("@@")
            if len(parts) == 3:
                log_id = parts[0]
                log_data = parts[1]
                error_message = parts[2]

                match = re.match(r'^(\S+) - (\S+) \[([^\]]+)\] "([^"]+)" (\d{3}) (\S+) "([^"]*)" "([^"]*)" "([^"]+)" (\S+)$', log_data)
                
                if match:
                    logs.append({
                        "log_type": "advertools_error",
                        "log_id": int(log_id) if log_id.isdigit() else log_id,
                        "ip": match.group(1),
                        "user": match.group(2),
                        "timestamp": match.group(3),
                        "request": match.group(4),
                        "status_code": int(match.group(5)),
                        "bytes": int(match.group(6)) if match.group(6).isdigit() else match.group(6),
                        "referer": match.group(7) if match.group(7) != '-' else None,
                        "user_agent": match.group(8) if match.group(8) != '-' else None,
                        "host": match.group(9),
                        "response_time": float(match.group(10)) if match.group(10).replace('.', '', 1).isdigit() else match.group(10),
                        "error_message": error_message
                    })
    return logs

def save_logs_to_mongodb(logs_data, collection):
    """
    파싱된 로그 데이터를 MongoDB에 저장합니다.
    """
    if logs_data:
        try:
            collection.insert_many(logs_data)
            print(f"{len(logs_data)}개의 로그 항목이 '{collection.name}' 컬렉션에 성공적으로 저장되었습니다.")
        except pymongo.errors.BulkWriteError as e:
            print(f"데이터 저장 중 오류 발생: {e.details}")
    else:
        print(f"저장할 로그 데이터가 없습니다. (대상 컬렉션: {collection.name})")

def delete_mongodb_database(client, db_name):
    """
    지정된 MongoDB 데이터베이스를 완전히 삭제합니다.
    """
    try:
        client.drop_database(db_name)
        print(f"\n'{db_name}' 데이터베이스와 모든 컬렉션/문서를 삭제했습니다.")
    except Exception as e:
        print(f"데이터베이스 삭제 중 오류가 발생했습니다: {e}")

## MongoDB 연결 및 데이터 저장

In [8]:
# --- MongoDB 연결 및 초기화 ---
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "log_database"

try:
    # MongoDB에 연결합니다.
    client = pymongo.MongoClient(MONGO_URI)
    db = client[DB_NAME]
    
    print(f"'{DB_NAME}' 데이터베이스에 연결했습니다.")

except pymongo.errors.ConnectionFailure as e:
    print(f"MongoDB에 연결할 수 없습니다. 서버가 실행 중인지 확인하세요: {e}")
except Exception as e:
    print(f"예상치 못한 오류가 발생했습니다: {e}")

'log_database' 데이터베이스에 연결했습니다.


## error_logs 컬렉션에 데이터 저장

In [9]:
# error_logs.txt 파일을 'error_logs' 컬렉션에 저장
ERROR_LOGS_COLLECTION = "error_logs"
error_logs_collection = db[ERROR_LOGS_COLLECTION]

print(f"\n'{ERROR_LOGS_COLLECTION}' 컬렉션에 데이터 저장 시작...")
try:
    error_logs_data = parse_error_logs("data/error_logs.txt")
    save_logs_to_mongodb(error_logs_data, error_logs_collection)
except FileNotFoundError:
    print("'data/error_logs.txt' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"'error_logs.txt' 처리 중 오류가 발생했습니다: {e}")


'error_logs' 컬렉션에 데이터 저장 시작...
11개의 로그 항목이 'error_logs' 컬렉션에 성공적으로 저장되었습니다.


## advertools_error_logs 컬렉션에 데이터 저장

In [10]:
# advertools_error_logs.txt 파일을 'advertools_error_logs' 컬렉션에 저장
ADVERTOOLS_LOGS_COLLECTION = "advertools_error_logs"
advertools_logs_collection = db[ADVERTOOLS_LOGS_COLLECTION]

print(f"\n'{ADVERTOOLS_LOGS_COLLECTION}' 컬렉션에 데이터 저장 시작...")
try:
    advertools_logs_data = parse_advertools_logs("data/advertools_error_logs.txt")
    save_logs_to_mongodb(advertools_logs_data, advertools_logs_collection)
except FileNotFoundError:
    print("'data/advertools_error_logs.txt' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"'advertools_error_logs.txt' 처리 중 오류가 발생했습니다: {e}")


'advertools_error_logs' 컬렉션에 데이터 저장 시작...
300개의 로그 항목이 'advertools_error_logs' 컬렉션에 성공적으로 저장되었습니다.


## error_logs 컬렉션 조회 예제

### 모든 문서 조회

In [11]:
documents = error_logs_collection.find().limit(5)
for doc in documents:
    print(doc)

{'_id': ObjectId('68a5646c6d0b2fb118f138da'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'status_code': 400}
{'_id': ObjectId('68a5646c6d0b2fb118f138db'), 'log_type': 'corrupted', 'message': 'BROKEN LINE WITHOUT FIELDS'}
{'_id': ObjectId('68a5646c6d0b2fb118f138dc'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Mobile/15E148 Safari/604.1', 'status_code': 302}
{'_id': ObjectId('68a5646c6d0b2fb118f138dd'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0', 'status_code': 201}
{'_id': ObjectId('68a5646c6d0b2fb118f138de'), 'log_type': 'corrupted', 'message': 'BROKEN LINE WITHOUT FIELDS'}


### 특정 Log Type(incomplete)를 가진 문서 조회

In [12]:
query = {"log_type": 'incomplete'}
documents = error_logs_collection.find(query)
for doc in documents:
    print(doc)

{'_id': ObjectId('68a5646c6d0b2fb118f138da'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'status_code': 400}
{'_id': ObjectId('68a5646c6d0b2fb118f138dc'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Mobile/15E148 Safari/604.1', 'status_code': 302}
{'_id': ObjectId('68a5646c6d0b2fb118f138dd'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0', 'status_code': 201}
{'_id': ObjectId('68a5646c6d0b2fb118f138e0'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'status_code': 503}


### 특정 log_type의 문서 개수 세기

In [13]:
count = error_logs_collection.count_documents({"log_type": "incomplete"})
print(f"총 {count}개의 'incomplete' 로그가 있습니다.")

총 4개의 'incomplete' 로그가 있습니다.


## advertools_error_logs 컬렉션 조회 예제

### 특정 에러 메시지를 가진 문서 조회

In [14]:
query = {"error_message": "list index out of range"}

# 조건에 맞는 문서의 총 개수
count = advertools_logs_collection.count_documents(query)
print(f"총 {count}개의 문서가 있습니다.")

# 조회된 문서의 상위 5개 출력
documents = advertools_logs_collection.find(query).limit(5)
for doc in documents:
    print(doc)

총 300개의 문서가 있습니다.
{'_id': ObjectId('68a5646c6d0b2fb118f138e5'), 'log_type': 'advertools_error', 'log_id': 300, 'ip': '10.0.0.5', 'user': 'dave', 'timestamp': '16/Aug/2025:12:33:16 +0000', 'request': 'POST /static/app.js HTTP/2.0', 'status_code': 403, 'bytes': 1273, 'referer': None, 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'host': 'api.example.com', 'response_time': 3.183, 'error_message': 'list index out of range'}
{'_id': ObjectId('68a5646c6d0b2fb118f138e6'), 'log_type': 'advertools_error', 'log_id': 301, 'ip': '52.79.152.10', 'user': '-', 'timestamp': '17/Aug/2025:13:49:39 +0000', 'request': 'PUT /api/v1/items/42?utm_source=newsletter&utm_campaign=4 HTTP/1.1', 'status_code': 301, 'bytes': 3995, 'referer': 'https://twitter.com/someone/status/1', 'user_agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0', 'host': 'example.com', 'response_time': 2.538, 'error

### 응답 시간(Response Time)이 1초 이상인 로그 조회

In [15]:
query = {"response_time": {"$gt": 1.0}}

# 조건에 맞는 문서의 총 개수
count = advertools_logs_collection.count_documents(query)
print(f"총 {count}개의 문서가 있습니다.")

# 조회된 문서의 상위 5개 출력
documents = advertools_logs_collection.find(query).limit(5)
for doc in documents:
    print(doc)

총 226개의 문서가 있습니다.
{'_id': ObjectId('68a5646c6d0b2fb118f138e5'), 'log_type': 'advertools_error', 'log_id': 300, 'ip': '10.0.0.5', 'user': 'dave', 'timestamp': '16/Aug/2025:12:33:16 +0000', 'request': 'POST /static/app.js HTTP/2.0', 'status_code': 403, 'bytes': 1273, 'referer': None, 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'host': 'api.example.com', 'response_time': 3.183, 'error_message': 'list index out of range'}
{'_id': ObjectId('68a5646c6d0b2fb118f138e6'), 'log_type': 'advertools_error', 'log_id': 301, 'ip': '52.79.152.10', 'user': '-', 'timestamp': '17/Aug/2025:13:49:39 +0000', 'request': 'PUT /api/v1/items/42?utm_source=newsletter&utm_campaign=4 HTTP/1.1', 'status_code': 301, 'bytes': 3995, 'referer': 'https://twitter.com/someone/status/1', 'user_agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0', 'host': 'example.com', 'response_time': 2.538, 'error

### 상태 코드와 IP 주소로 필터링하여 조회

In [16]:
query = {"status_code": 403, "ip": "10.0.0.5"}

# 조건에 맞는 문서의 총 개수
count = advertools_logs_collection.count_documents(query)
print(f"총 {count}개의 문서가 있습니다.")

# 조회된 문서의 상위 3개 출력
documents = advertools_logs_collection.find(query).limit(3)
for doc in documents:
    print(doc)

총 7개의 문서가 있습니다.
{'_id': ObjectId('68a5646c6d0b2fb118f138e5'), 'log_type': 'advertools_error', 'log_id': 300, 'ip': '10.0.0.5', 'user': 'dave', 'timestamp': '16/Aug/2025:12:33:16 +0000', 'request': 'POST /static/app.js HTTP/2.0', 'status_code': 403, 'bytes': 1273, 'referer': None, 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'host': 'api.example.com', 'response_time': 3.183, 'error_message': 'list index out of range'}
{'_id': ObjectId('68a5646c6d0b2fb118f13906'), 'log_type': 'advertools_error', 'log_id': 333, 'ip': '10.0.0.5', 'user': 'dave', 'timestamp': '16/Aug/2025:14:35:54 +0000', 'request': 'PATCH /search?q=python HTTP/1.1', 'status_code': 403, 'bytes': 2087, 'referer': 'https://www.bing.com/', 'user_agent': 'curl/8.6.0', 'host': 'example.com', 'response_time': 2.792, 'error_message': 'list index out of range'}
{'_id': ObjectId('68a5646c6d0b2fb118f1390e'), 'log_type': 'advertools_error', 'log_

### 모든 문서의 총 개수

In [17]:
print("\n컬렉션 내 모든 문서의 총 개수")
total_count = advertools_logs_collection.count_documents({})
print(f"'advertools_error_logs' 컬렉션에 총 {total_count}개의 문서가 저장되어 있습니다.")


컬렉션 내 모든 문서의 총 개수
'advertools_error_logs' 컬렉션에 총 300개의 문서가 저장되어 있습니다.


## 수정 예제

### 수정할 원본 문서 조회

In [18]:
import pymongo

# 아직 안 닫음
# MONGO_URI = "mongodb://localhost:27017/" 
# client = pymongo.MongoClient(MONGO_URI)

DB_NAME = "log_database"
ERROR_LOGS_COLLECTION = "error_logs"

db = client[DB_NAME]
error_logs_collection = db[ERROR_LOGS_COLLECTION]

print("수정할 'incomplete' 로그 원본을 조회합니다.")
incomplete_doc = error_logs_collection.find_one({"log_type": "incomplete"})

if incomplete_doc:
    print(f"원본 문서: {incomplete_doc}")
else:
    print("수정할 'incomplete' 타입의 문서를 찾을 수 없습니다.")

수정할 'incomplete' 로그 원본을 조회합니다.
원본 문서: {'_id': ObjectId('68a5646c6d0b2fb118f138da'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'status_code': 400}


### 원본 문서에 '수정됨' 플래그 추가

In [19]:
if incomplete_doc:
    original_doc_id = incomplete_doc['_id']
    
    # 원본 문서에 수정 플래그 추가
    error_logs_collection.update_one(
        {"_id": original_doc_id},
        {"$set": {"is_modified": True, "note": "이 문서는 수정본의 원본입니다."}}
    )
    print("\n 원본 문서에 'is_modified' 플래그를 추가했습니다.")
    
    # 변경된 원본 문서 확인
    updated_original_doc = error_logs_collection.find_one({"_id": original_doc_id})
    print(f"수정된 원본 문서: {updated_original_doc}")


 원본 문서에 'is_modified' 플래그를 추가했습니다.
수정된 원본 문서: {'_id': ObjectId('68a5646c6d0b2fb118f138da'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'status_code': 400, 'is_modified': True, 'note': '이 문서는 수정본의 원본입니다.'}


### 새로운 필드를 추가하여 수정본 저장

In [20]:
if incomplete_doc:
    # 수정된 데이터를 포함하는 새로운 문서 생성
    modified_doc = incomplete_doc.copy()
    modified_doc.pop('_id') # 새 문서이므로 _id 필드 제거
    
    modified_doc.update({
        "log_type": "incomplete_modified", # 수정본임을 나타내는 새로운 타입
        "ip": "192.168.1.100",           # 새로운 IP 정보 추가
        "timestamp": "16/Aug/2025:12:33:16 +0000", # 새로운 타임스탬프 추가
        "source_doc_id": original_doc_id # 원본 문서의 _id를 참조
    })
    
    # 수정된 문서를 컬렉션에 삽입
    result = error_logs_collection.insert_one(modified_doc)
    print(f"\n새로운 수정본 문서를 저장했습니다. _id: {result.inserted_id}")
    
    # 저장된 수정본 문서 확인
    newly_inserted_doc = error_logs_collection.find_one({"_id": result.inserted_id})
    print(f"새로 저장된 수정본: {newly_inserted_doc}")


새로운 수정본 문서를 저장했습니다. _id: 68a5646c6d0b2fb118f13a11
새로 저장된 수정본: {'_id': ObjectId('68a5646c6d0b2fb118f13a11'), 'log_type': 'incomplete_modified', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'status_code': 400, 'ip': '192.168.1.100', 'timestamp': '16/Aug/2025:12:33:16 +0000', 'source_doc_id': ObjectId('68a5646c6d0b2fb118f138da')}


### 원본문서와 수정본 문서 찾기

In [21]:
# 원본 문서 찾기 (수정 플래그가 있는 문서)
print("원본 문서 (is_modified: True)를 찾습니다.")
original_doc = error_logs_collection.find_one({"log_type": "incomplete", "is_modified": True})

# 수정본 문서 찾기 (원본의 _id를 참조하는 문서)
print("\n수정본 문서를 찾습니다.")
if original_doc:
    modified_doc = error_logs_collection.find_one({"log_type": "incomplete_modified", "source_doc_id": original_doc['_id']})
else:
    modified_doc = None

if original_doc and modified_doc:
    print("원본과 수정본 문서를 모두 찾았습니다.")
else:
    print("원본 또는 수정본 문서를 찾을 수 없습니다. 이전 단계를 다시 확인하세요.")

원본 문서 (is_modified: True)를 찾습니다.

수정본 문서를 찾습니다.
원본과 수정본 문서를 모두 찾았습니다.


In [22]:
if original_doc and modified_doc:
    print("\n--- 원본 문서 ---")
    print(original_doc)
    
    print("\n--- 수정본 문서 ---")
    print(modified_doc)
    
    print("\n--- 주요 필드 비교 ---")
    
    # 1. log_type 필드 비교
    print(f"로그 타입: 원본='{original_doc.get('log_type')}', 수정본='{modified_doc.get('log_type')}'")

    # 2. IP 필드 추가 여부 확인
    if 'ip' in modified_doc and 'ip' not in original_doc:
        print(f"IP 필드: 원본에 없음, 수정본에 추가됨 ('{modified_doc['ip']}')")
    
    # 3. Timestamp 필드 추가 여부 확인
    if 'timestamp' in modified_doc and 'timestamp' not in original_doc:
        print(f"타임스탬프 필드: 원본에 없음, 수정본에 추가됨 ('{modified_doc['timestamp']}')")

    # 4. _id 필드 참조 확인
    print(f"원본 문서 ID: {original_doc.get('_id')}")
    print(f"수정본의 원본 참조 ID: {modified_doc.get('source_doc_id')}")


--- 원본 문서 ---
{'_id': ObjectId('68a5646c6d0b2fb118f138da'), 'log_type': 'incomplete', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'status_code': 400, 'is_modified': True, 'note': '이 문서는 수정본의 원본입니다.'}

--- 수정본 문서 ---
{'_id': ObjectId('68a5646c6d0b2fb118f13a11'), 'log_type': 'incomplete_modified', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15', 'status_code': 400, 'ip': '192.168.1.100', 'timestamp': '16/Aug/2025:12:33:16 +0000', 'source_doc_id': ObjectId('68a5646c6d0b2fb118f138da')}

--- 주요 필드 비교 ---
로그 타입: 원본='incomplete', 수정본='incomplete_modified'
IP 필드: 원본에 없음, 수정본에 추가됨 ('192.168.1.100')
타임스탬프 필드: 원본에 없음, 수정본에 추가됨 ('16/Aug/2025:12:33:16 +0000')
원본 문서 ID: 68a5646c6d0b2fb118f138da
수정본의 원본 참조 ID: 68a5646c6d0b2fb118f138da


## 문서, 컬렉션, DB 삭제 예제

### MongoDB 연결 및 컬렉션 지정

In [23]:
import pymongo

DB_NAME = "log_database"
ERROR_LOGS_COLLECTION = "error_logs"

db = client[DB_NAME]
error_logs_collection = db[ERROR_LOGS_COLLECTION]

### error_logs 컬렉션에서 수정본 문서 삭제

In [24]:
query = {"log_type": "incomplete_modified"}

try:
    result = error_logs_collection.delete_many(query)
    print(f"{result.deleted_count}개의 수정본 문서가 삭제되었습니다.")
    
    # 삭제 후 남은 문서 개수 확인
    remaining_count = error_logs_collection.count_documents({})
    print(f"현재 'error_logs' 컬렉션에 남아있는 문서 개수: {remaining_count}")
    
except Exception as e:
    print(f"수정본 문서 삭제 중 오류 발생: {e}")

1개의 수정본 문서가 삭제되었습니다.
현재 'error_logs' 컬렉션에 남아있는 문서 개수: 11


### error_logs 컬렉션 삭제

In [25]:
print("\n'error_logs' 컬렉션을 삭제합니다.")

try:
    error_logs_collection.drop()
    print("'error_logs' 컬렉션이 삭제되었습니다.")
    
except Exception as e:
    print(f"컬렉션 삭제 중 오류 발생: {e}")


'error_logs' 컬렉션을 삭제합니다.
'error_logs' 컬렉션이 삭제되었습니다.


### 데이터베이스 삭제 및 연결 종료 

In [26]:
try:
    client.drop_database(DB_NAME)
    print(f"'{DB_NAME}' 데이터베이스가 완전히 삭제되었습니다.")
    
except Exception as e:
    print(f"데이터베이스 삭제 중 오류 발생: {e}")

# MongoDB 연결 종료
client.close()
print("\n 모든 작업이 완료되었습니다. MongoDB 연결을 종료합니다.")

'log_database' 데이터베이스가 완전히 삭제되었습니다.

 모든 작업이 완료되었습니다. MongoDB 연결을 종료합니다.


### advertools_error_logs에 새로운 문서 추가

In [32]:
import pymongo
from datetime import datetime

MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "log_database"
ADVERTOOLS_LOGS_COLLECTION = "advertools_error_logs"

client = pymongo.MongoClient(MONGO_URI)
db = client[DB_NAME]
advertools_logs_collection = db[ADVERTOOLS_LOGS_COLLECTION]

In [34]:
# 현재 시간을 'advertools' 로그 형식에 맞게 변환
current_timestamp = datetime.now().strftime("%d/%b/%Y:%H:%M:%S +0000")

# 새로운 임의의 로그 문서
new_log = {
    "log_type": "advertools_error",
    "log_id": 9999,
    "ip": "203.0.113.42",
    "user": "test_user",
    "timestamp": current_timestamp,
    "request": "GET /api/data/123 HTTP/2.0",
    "status_code": 500,
    "bytes": 512,
    "referer": "-",
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "host": "api.example.com",
    "response_time": 0.521,
    "error_message": "Internal Server Error: Database connection failed"
}

print("새로운 임의의 로그 문서를 삽입합니다.")
try:
    result = advertools_logs_collection.insert_one(new_log)
    print(f"문서가 성공적으로 삽입되었습니다. _id: {result.inserted_id}")

    # 삽입된 문서가 올바르게 저장되었는지 확인
    inserted_doc = advertools_logs_collection.find_one({"_id": result.inserted_id})
    print("\n삽입된 문서:")
    print(inserted_doc)

except Exception as e:
    print(f"문서 삽입 중 오류 발생: {e}")

새로운 임의의 로그 문서를 삽입합니다.
문서가 성공적으로 삽입되었습니다. _id: 68a56ba86d0b2fb118f13b44

삽입된 문서:
{'_id': ObjectId('68a56ba86d0b2fb118f13b44'), 'log_type': 'advertools_error', 'log_id': 9999, 'ip': '203.0.113.42', 'user': 'test_user', 'timestamp': '20/Aug/2025:15:31:04 +0000', 'request': 'GET /api/data/123 HTTP/2.0', 'status_code': 500, 'bytes': 512, 'referer': '-', 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36', 'host': 'api.example.com', 'response_time': 0.521, 'error_message': 'Internal Server Error: Database connection failed'}


### 특정 기간 동안의 로그 삭제하기

In [35]:
import pymongo
import re
from datetime import datetime

# MongoDB 연결 및 컬렉션 지정
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "log_database"
ADVERTOOLS_LOGS_COLLECTION = "advertools_error_logs"

client = pymongo.MongoClient(MONGO_URI)
db = client[DB_NAME]
advertools_logs_collection = db[ADVERTOOLS_LOGS_COLLECTION]

# --- 1. 컬렉션 삭제 (재시작) ---
print("🧹 기존 'advertools_error_logs' 컬렉션을 삭제합니다.")
advertools_logs_collection.drop()
print("컬렉션이 성공적으로 삭제되었습니다.")

# --- 2. 데이터 파싱 함수 정의 ---
def parse_advertools_logs(file_path):
    """
    advertools_error_logs.txt 파일을 파싱하여 JSON 문서 리스트로 반환합니다.
    이때 timestamp 필드는 datetime 객체로 변환됩니다.
    """
    logs = []
    date_format = "%d/%b/%Y:%H:%M:%S +0000"

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.split("@@")
            if len(parts) == 3:
                log_id = parts[0]
                log_data = parts[1]
                error_message = parts[2]

                match = re.match(r'^(\S+) - (\S+) \[([^\]]+)\] "([^"]+)" (\d{3}) (\S+) "([^"]*)" "([^"]*)" "([^"]+)" (\S+)$', log_data)
                
                if match:
                    timestamp_str = match.group(3)
                    
                    try:
                        timestamp_obj = datetime.strptime(timestamp_str, date_format)
                    except ValueError:
                        timestamp_obj = timestamp_str

                    logs.append({
                        "log_type": "advertools_error",
                        "log_id": int(log_id) if log_id.isdigit() else log_id,
                        "ip": match.group(1),
                        "user": match.group(2),
                        "timestamp": timestamp_obj, # datetime 객체로 저장
                        "request": match.group(4),
                        "status_code": int(match.group(5)),
                        "bytes": int(match.group(6)) if match.group(6).isdigit() else match.group(6),
                        "referer": match.group(7) if match.group(7) != '-' else None,
                        "user_agent": match.group(8) if match.group(8) != '-' else None,
                        "host": match.group(9),
                        "response_time": float(match.group(10)) if match.group(10).replace('.', '', 1).isdigit() else match.group(10),
                        "error_message": error_message
                    })
    return logs

# --- 3. 데이터 재삽입 ---
print("\n './data/advertools_error_logs.txt' 파일을 파싱하여 새로운 데이터를 삽입합니다.")
file_path = "./data/advertools_error_logs.txt"  # 실제 파일 경로로 변경해주세요.
advertools_logs_data = parse_advertools_logs(file_path)

if advertools_logs_data:
    advertools_logs_collection.insert_many(advertools_logs_data)
    print(f"{len(advertools_logs_data)}개의 문서가 성공적으로 삽입되었습니다.")
    print(f"현재 'advertools_error_logs' 컬렉션의 문서 개수: {advertools_logs_collection.count_documents({})}")
else:
    print("파싱할 데이터를 찾을 수 없습니다. 파일 경로를 확인해주세요.")

# --- 4. datetime 객체로 날짜 기반 삭제 ---
print("\n특정 기간의 로그를 삭제합니다.")

# 삭제 기간 정의: 2025년 8월 17일 10시 00분부터 2025년 8월 19일 15시 00분까지
start_date = datetime(2025, 8, 17, 10, 0, 0)
end_date = datetime(2025, 8, 19, 15, 0, 0)

# 삭제 쿼리
query = {
    "timestamp": {
        "$gte": start_date,
        "$lt": end_date
    }
}

try:
    result = advertools_logs_collection.delete_many(query)
    print(f"{result.deleted_count}개의 문서가 성공적으로 삭제되었습니다.")
    
    # 삭제 후 남은 문서 개수 확인
    remaining_count = advertools_logs_collection.count_documents({})
    print(f"현재 'advertools_error_logs' 컬렉션에 남아있는 문서 개수: {remaining_count}")

except Exception as e:
    print(f"문서 삭제 중 오류 발생: {e}")

# MongoDB 연결 종료
client.close()
print("\n모든 작업이 완료되었습니다. MongoDB 연결을 종료합니다.")

🧹 기존 'advertools_error_logs' 컬렉션을 삭제합니다.
컬렉션이 성공적으로 삭제되었습니다.

 './data/advertools_error_logs.txt' 파일을 파싱하여 새로운 데이터를 삽입합니다.
300개의 문서가 성공적으로 삽입되었습니다.
현재 'advertools_error_logs' 컬렉션의 문서 개수: 300

특정 기간의 로그를 삭제합니다.
127개의 문서가 성공적으로 삭제되었습니다.
현재 'advertools_error_logs' 컬렉션에 남아있는 문서 개수: 173

모든 작업이 완료되었습니다. MongoDB 연결을 종료합니다.


## 데이터베이스 삭제 및 연결 종료 

In [36]:
import pymongo

MONGO_URI = "mongodb://localhost:27017/"

try:
    # Connect to MongoDB client
    client = pymongo.MongoClient(MONGO_URI)
    database_names = client.list_database_names()
    print(f"\nCurrent databases: {database_names}")
    
except pymongo.errors.ConnectionFailure as e:
    print(f"Connection error: {e}")


Current databases: ['admin', 'config', 'local', 'log_database']


In [37]:
# System databases to be skipped
system_databases = ["admin", "config", "local"]

if database_names:
    print("\n Deleting all non-system databases...")
    for db_name in database_names:
        if db_name not in system_databases:
            client.drop_database(db_name)
            print(f" Database '{db_name}' has been deleted.")
        else:
            print(f" Skipping system database '{db_name}'.")

    # Verify deletion by listing databases again
    remaining_databases = client.list_database_names()
    print(f"\n Databases remaining: {remaining_databases}")

# Close the connection
client.close()
print("\n Connection closed. All operations completed.")


 Deleting all non-system databases...
 Skipping system database 'admin'.
 Skipping system database 'config'.
 Skipping system database 'local'.
 Database 'log_database' has been deleted.

 Databases remaining: ['admin', 'config', 'local']

 Connection closed. All operations completed.
